In [ ]:
%load_ext autoreload
%autoreload 2
from spyglass.ripple.v1 import RippleTimesV1, RippleParameters
from spyglass.common import interval_list_intersect
from spyglass.decoding.v1.clusterless import ClusterlessDecodingV1
from spyglass.lfp import LFPOutput

import numpy as np
from scipy.stats import ttest_rel
import statsmodels.stats as sm
from statsmodels.stats import proportion

import os
import matplotlib.pyplot as plt

from ms_stim_analysis.AnalysisTables.ms_opto_stim_protocol import OptoStimProtocol
from ms_stim_analysis.Analysis.utils import filter_opto_data, violin_scatter
from ms_stim_analysis.Style.style_guide import animal_style, interval_style
from ms_stim_analysis.AnalysisTables.decoding_tables import ClusterlessAheadBehindDistance, BadDecodes
from ms_stim_analysis.AnalysisTables.place_fields import DecodesToCoveredTrack

from ms_stim_analysis.AnalysisTables.decoding_tables import RippleClusterlessDecodeAnalysisSelection, RippleClusterlessDecodeAnalysis


from spyglass.position.v1 import TrodesPosV1
from spyglass.position.position_merge import PositionOutput
from spyglass.lfp.analysis.v1 import LFPBandV1

# Populate

In [ ]:
dlc_pos = True
acausal = False

for animal in animal_style.index:
    dataset_key = {"animal": animal, "track_type": "early_wtrack"}
    dataset = filter_opto_data(dataset_key)

    ripple_class = []
    for key in dataset.fetch("KEY"):
        clusterless_key = {
            "nwb_file_name": key["nwb_file_name"],
            "encoding_interval": key["interval_list_name"],
        }
        query = ClusterlessDecodingV1() & clusterless_key
        if dlc_pos:
            query = query & "position_group_name LIKE 'dlc%'"
        else:
            query = query - "position_group_name LIKE 'dlc%'"
        if not query:
            continue
        clusterless_key = query.fetch1("KEY")
        if not ClusterlessAheadBehindDistance() & clusterless_key:
            continue
        query = (
            RippleTimesV1()
            & {
                "nwb_file_name": key["nwb_file_name"],
            }
            & "target_interval_list_name LIKE '%{}%'".format(key["interval_list_name"])
        )
        if dlc_pos:
            query = query & PositionOutput.DLCPosV1().proj(pos_merge_id="merge_id")
        else:
            query & PositionOutput.TrodesPosV1().proj(pos_merge_id="merge_id")

        ripple_key = query.fetch1("KEY")
        analysis_key = {**ripple_key, **clusterless_key, "acausal": acausal}
        RippleClusterlessDecodeAnalysisSelection().insert1(
            analysis_key, skip_duplicates=True
        )

In [ ]:
RippleClusterlessDecodeAnalysis().populate(processes=16, suppress_errors=True)